In [1]:
import os
import time
import copy
import glob 
import torch
import random
import numpy as np
import matplotlib
import torchvision
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from PIL import Image as im
from model import Gauss2D

In [2]:
show        = True 
pretrain    = False
data_dir    = 'data'
filename    = 'model_out_gauss.pth'
batch_size  = 20       #Number of samples in each batch
num_workers = 4
epoch_num   = 5        #Number of epochs to train the network
lr          = 0.01     # Learning rate

In [3]:
data_transforms = {
    'train': transforms.ToTensor(),
    'val': transforms.ToTensor()
}
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=num_workers)
               for x in ['train', 'val']}
print(len(dataloaders['train']))
print(len(dataloaders['val']))

180
20


In [4]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) 

cpu


In [5]:
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

#Gauss2D.imshow(out,title=[class_names[x] for x in classes], fname='initial.jpg',show=False)


model_ft = models.resnet18(pretrained=pretrain)
if torch.cuda.device_count() > 1: 
    print("Using ", torch.cuda.device_count(),"gpus!")
    model_ft = nn.DataParallel(model_ft) #
    
model_ft=model_ft.to(device)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=lr, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=10, gamma=0.1)
model_ft = Gauss2D.train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=epoch_num, dataloaders=dataloaders, device=device,dataset_sizes=dataset_sizes)
torch.save(model_ft, filename)

#Gauss2D.visualize_model(model_ft,num_images=4,device=device,dataloaders=dataloaders,class_names=class_names)

Epoch 0/4
----------
train Loss: 0.3008 Acc: 0.9256
val Loss: 0.1055 Acc: 0.9548

Epoch 1/4
----------
train Loss: 0.1622 Acc: 0.9542
val Loss: 0.1079 Acc: 0.9724

Epoch 2/4
----------
train Loss: 0.1325 Acc: 0.9539
val Loss: 0.0811 Acc: 0.9673

Epoch 3/4
----------
train Loss: 0.1139 Acc: 0.9575
val Loss: 0.0886 Acc: 0.9673

Epoch 4/4
----------
train Loss: 0.1101 Acc: 0.9619
val Loss: 0.0534 Acc: 0.9749

Training complete in 3m 17s
Best val Acc: 0.974874


In [7]:
from sklearn.metrics import confusion_matrix
import seaborn as sns 
import pandas as pd 

nb_classes = 2

# Initialize the prediction and label lists(tensors)
predlist=torch.zeros(0,dtype=torch.long, device='cpu')
lbllist=torch.zeros(0,dtype=torch.long, device='cpu')

with torch.no_grad():
    for i, (inputs, classes) in enumerate(dataloaders['val']):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)

        # Append batch prediction results
        predlist=torch.cat([predlist,preds.view(-1).cpu()])
        lbllist=torch.cat([lbllist,classes.view(-1).cpu()])

# Confusion matrix
conf_mat=confusion_matrix(lbllist.numpy(), predlist.numpy())
print(conf_mat)

# Per-class accuracy
class_accuracy=100*conf_mat.diagonal()/conf_mat.sum(1)
print(class_accuracy)
labels = ['one','two']
ax= plt.subplot()


sns.heatmap(conf_mat, annot=True)
class_names = np.asarray(labels)
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_xticklabels(class_names.flatten())
ax.set_yticklabels(class_names.flatten())
ax.set_title('Confusion Matrix')

ModuleNotFoundError: No module named 'seaborn'